In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import numpy as np
import torch
from PIL import Image
import numpy as np
#from IPython.display import Image

import sys
sys.path.append('/content/drive/MyDrive/detection')
import utils

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [4]:
imgs = list(sorted(os.listdir('/content/drive/MyDrive/maskrcnn_data/PNGImages/PNG')))
masks = list(sorted(os.listdir('/content/drive/MyDrive/maskrcnn_data/PedMasks/Mask')))

print(len(imgs), len(masks))

KeyboardInterrupt: ignored

In [5]:
import os
import numpy as np
import torch
from PIL import Image

class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir('/content/drive/MyDrive/maskrcnn_data/PNGImages/PNG')))
        self.masks = list(sorted(os.listdir('/content/drive/MyDrive/maskrcnn_data/PedMasks/Mask')))

        rm_name_list = ['C201_S1_I6_R', 'C487_S1_I14_L', 'C247_S2_I13_L', 'C205_S1_I12_R', 'C495_S1_I2_L', 'C205_S1_I6_R']
        rm_list = []
        for rm_name in rm_name_list:
          rm_list.extend([rm_name + '.jpg', rm_name + '.png'])
        
        self.imgs = [i for i in self.imgs if i not in rm_list]
        self.masks = [i for i in self.masks if i not in rm_list]

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join('/content/drive/MyDrive/maskrcnn_data/PNGImages/PNG', self.imgs[idx])
        mask_path = os.path.join('/content/drive/MyDrive/maskrcnn_data/PedMasks/Mask', self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        #area = torch.tensor([0,0,0])
        try:
          area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
          #print(area, type(area))
        except:
          print(self.imgs[idx], self.masks[idx], image_id, boxes)          
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)
        return len(self.imgs)

In [6]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [7]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [8]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [9]:
from torchvision import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

KeyboardInterrupt: ignored

In [ ]:
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

for i in range(0, 500):
  train_features, train_labels = next(iter(data_loader))
  for t in train_labels:
    if len(t['boxes']) == 0:
      print(len(t['boxes']))

C205_S1_I6_R.jpg C205_S1_I6_R.png tensor([2575]) tensor([])
0
C247_S2_I13_L.jpg C247_S2_I13_L.png tensor([3548]) tensor([])


KeyboardInterrupt: ignored

In [ ]:
from engine import train_one_epoch, evaluate
import utils

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - background and person
num_classes = 4
# use our dataset and defined transformations
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))
# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:8500])
dataset_test = torch.utils.data.Subset(dataset_test, indices[8500:])
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)
# get the model using our helper function
model = get_model_instance_segmentation(num_classes)
# move model to the right device
model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)
# let's train it for 10 epochs
num_epochs = 10
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)
print("That's it!")



Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

Epoch: [0]  [   0/4250]  eta: 10:52:27  lr: 0.000010  loss: 3.7453 (3.7453)  loss_classifier: 1.5112 (1.5112)  loss_box_reg: 0.0732 (0.0732)  loss_mask: 1.4929 (1.4929)  loss_objectness: 0.6109 (0.6109)  loss_rpn_box_reg: 0.0571 (0.0571)  time: 9.2111  data: 1.5772  max mem: 2227
Epoch: [0]  [  10/4250]  eta: 1:10:35  lr: 0.000060  loss: 3.5082 (3.3987)  loss_classifier: 1.3793 (1.3642)  loss_box_reg: 0.1615 (0.1534)  loss_mask: 1.5030 (1.4881)  loss_objectness: 0.3237 (0.3475)  loss_rpn_box_reg: 0.0491 (0.0455)  time: 0.9990  data: 0.2084  max mem: 2518
Epoch: [0]  [  20/4250]  eta: 0:45:57  lr: 0.000110  loss: 2.2207 (2.6505)  loss_classifier: 0.9489 (0.9961)  loss_box_reg: 0.1615 (0.1567)  loss_mask: 0.9125 (1.1793)  loss_objectness: 0.2383 (0.2786)  loss_rpn_box_reg: 0.0353 (0.0397)  time: 0.2238  data: 0.1191  max mem: 2519
Epoch: [0]  [  30/4250]  eta: 0:38:33  lr: 0.000160  loss: 1.3535 (2.2001)  loss_classifier: 0.3339 (0.7641)  loss_box_reg: 0.1775 (0.1668)  loss_mask: 0.7544 

KeyboardInterrupt: ignored

In [ ]:
!pip install sgmllib3k

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
torch.save(model, '/content/drive/MyDrive/tt.pth')

In [10]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - background and person
num_classes = 4
# get the model using our helper function
model = torch.load("/content/drive/MyDrive/tt.pth")
model.to(torch.device('cpu'))

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [11]:
import os
import numpy as np
import torch
from PIL import Image

class PennFudanDataset2(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir('/content/drive/MyDrive/maskrcnn_data/PNGImages/PNG_trial')))
        self.masks = list(sorted(os.listdir('/content/drive/MyDrive/maskrcnn_data/PedMasks/Mask_trial')))

        rm_name_list = ['C201_S1_I6_R', 'C487_S1_I14_L', 'C247_S2_I13_L', 'C205_S1_I12_R', 'C495_S1_I2_L', 'C205_S1_I6_R']
        rm_list = []
        for rm_name in rm_name_list:
          rm_list.extend([rm_name + '.jpg', rm_name + '.png'])
        
        self.imgs = [i for i in self.imgs if i not in rm_list]
        self.masks = [i for i in self.masks if i not in rm_list]

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join('/content/drive/MyDrive/maskrcnn_data/PNGImages/PNG_trial', self.imgs[idx])
        mask_path = os.path.join('/content/drive/MyDrive/maskrcnn_data/PedMasks/Mask_trial', self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        #area = torch.tensor([0,0,0])
        try:
          area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
          #print(area, type(area))
        except:
          print(self.imgs[idx], self.masks[idx], image_id, boxes)          
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [14]:
dataset = PennFudanDataset('PennFudanPed', get_transform(train=False))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
#output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(images)           # Returns predictions
print(predictions)

from torchvision.utils import save_image


saver = predictions[0]['masks'][2]
save_image(saver, '/content/drive/MyDrive/test.png')

[{'boxes': tensor([[ 273.0446,   95.9690, 1044.8710,  286.8095],
        [ 402.1735,  391.7085,  612.7361,  534.8038],
        [ 408.0504,  420.0937,  753.0295,  538.4097],
        [ 574.0043,  423.7237,  775.8379,  537.1044],
        [ 523.2474,  406.8140,  717.1691,  531.0329],
        [ 770.2768,  387.4196,  964.5133,  532.5756],
        [ 657.4044,  406.1480,  911.3472,  537.0043],
        [ 627.0947,  401.6800,  799.7316,  519.5389],
        [ 847.0899,  381.8322, 1042.4185,  536.5903]],
       grad_fn=<StackBackward0>), 'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1]), 'scores': tensor([0.9989, 0.9816, 0.9761, 0.7475, 0.4310, 0.3869, 0.3228, 0.0746, 0.0608],
       grad_fn=<IndexBackward0>), 'masks': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0